# 项目：评估和清理英国电商公司销售数据

## 分析目标

此数据分析的目的是，根据市场销售数据，挖掘畅销产品，以便制定更有效的市场策略来提升营收。

本实战项目的目的在于练习评估数据干净和整洁度，并且基于评估结果，对数据进行清洗，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了一家英国在线零售公司在2010年12月1日至2011年12月9日期间的所有交易情况，涵盖了该公司在全球不同国家和地区的业务数据。该公司主要销售覆盖各个场景的礼品，包括但不限于生日礼品、结婚纪念品、圣诞礼品等等。该公司的客户群体主要包括批发商和个人消费者，其中批发商占据了相当大的比例。

数据每列的含义如下：
- `InvoiceNo`: 发票号码。6位数，作为交易的唯一标识符。如果这个代码以字母“c”开头，表示这笔交易被取消。
- `StockCode`: 产品代码。5位数，作为产品的唯一标识符。
- `Description`: 产品名称。
- `Quantity`: 产品在交易中的数量。
- `InvoiceDate`: 发票日期和时间。交易发生的日期和时间。
- `UnitPrice`: 单价。价格单位为英镑（£）。
- `CustomerID`: 客户编号。5位数，作为客户的唯一标识符。
- `Country`: 国家名称。客户所居住的国家的名称。

## 读取数据

In [2]:
import pandas as pd

df = pd.read_csv('e_commerce.csv')
df.head()
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


## 评估数据
在这一步，我们主要从两个方面开始评估：结构和内容。即整齐度和干净度 \
结构性问题：不符合每列是一个变量，每行是一个观察值，每个单元格个是一个值的数据 \
内容行问题：包括丢失数据、重复数据、无效数据

In [5]:
# df.info(): 查看数据类型、非空值数量
# df.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


### 评估结果
内容评估：
Description & CustomerID 缺少字段\
CustomerID 数据类型应该是Object，而不是float\
InvoiceDate：应该是date，而不是object

In [7]:
# 进一步评估Description为空的结果
df[df["Description"].isnull() & (df["UnitPrice"] == 0)]
# 评估结论：Description为空的数据没有实际  意义，可以删除

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
622,536414,22139,NaN,56,12/1/2010 11:52,0.0,NaN,United Kingdom
1970,536545,21134,NaN,1,12/1/2010 14:32,0.0,NaN,United Kingdom
1971,536546,22145,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1972,536547,37509,NaN,1,12/1/2010 14:33,0.0,NaN,United Kingdom
1987,536549,85226A,NaN,1,12/1/2010 14:34,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
535322,581199,84581,NaN,-2,12/7/2011 18:26,0.0,NaN,United Kingdom
535326,581203,23406,NaN,15,12/7/2011 18:31,0.0,NaN,United Kingdom
535332,581209,21620,NaN,6,12/7/2011 18:35,0.0,NaN,United Kingdom
536981,581234,72817,NaN,27,12/8/2011 10:33,0.0,NaN,United Kingdom


In [11]:
# df.descriptive(): 查看数值型数据的统计信息
df.describe()
# 发现 quantity 产品在交易中的数量 and UnitPrice: 单价。价格单位为英镑（£）。 存在<0的情况 检查这些数据
df[df["Quantity"] < 0]
# 发现这些数据的发票号都是C开头的 是代表取消交易
df[(df["Quantity"] < 0) & df["InvoiceNo"]]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


## 清理数据

### 根据前方的评估结果可知，我们需要对数据进行清理
1、把InvoiceDate变量 数据类型转换为日期
2、CustomerID 变量 转换成字符串
3、Description 为空的记录删除
4、把country变量值 USA 替换为 united states
5、country变量值 UK 替换为 united kingdom
6、删除Quantity小于0
7、unitprice < 0 的记录删除

In [6]:
# 将df copy出来一份
clean_data = df.copy()
# 1、把InvoiceDate变量 数据类型转换为日期
clean_data["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])
clean_data["InvoiceDate"]

0        2010-12-01 08:26:00
1        2010-12-01 08:26:00
2        2010-12-01 08:26:00
3        2010-12-01 08:26:00
4        2010-12-01 08:26:00
                 ...        
541904   2011-12-09 12:50:00
541905   2011-12-09 12:50:00
541906   2011-12-09 12:50:00
541907   2011-12-09 12:50:00
541908   2011-12-09 12:50:00
Name: InvoiceDate, Length: 541909, dtype: datetime64[ns]

In [10]:
# 2、CustomerID 变量 转换成字符串,并且去掉小数位
clean_data["CustomerID"] = df["CustomerID"].astype(str).str.slice(0, -2)
clean_data["CustomerID"]

0         17850
1         17850
2         17850
3         17850
4         17850
          ...  
541904    12680
541905    12680
541906    12680
541907    12680
541908    12680
Name: CustomerID, Length: 541909, dtype: object

In [11]:
# Description 为空的记录删除
df["Description"].isnull().sum()
clean_data = clean_data.drop(clean_data[(clean_data["Description"].isnull())].index)
clean_data["Description"].isnull().sum()

0

In [32]:
# 把country变量值 USA 替换为 united states
# country变量值 UK 替换为 united kingdom
clean_data['Country'] = clean_data['Country'].replace('USA', 'united states')
clean_data['Country'] = clean_data['Country'].replace('UK', 'united kingdom')
clean_data['Country'] = clean_data['Country'].replace('U.K.', 'united kingdom')

clean_data[(clean_data['Country'] == "USA") | (clean_data['Country'] == "UK") | (clean_data['Country'] == "U.K.")]


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [25]:
# 删除Quantity小于0
clean_data = clean_data[clean_data["Quantity"] >= 0]
clean_data[clean_data["Quantity"] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [27]:
# 7、unitprice < 0 的记录删除
clean_data = clean_data[clean_data["UnitPrice"] >= 0]
clean_data[clean_data["Quantity"] <= 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


## 保存清理后的数据

In [34]:
clean_data.info()
clean_data.to_csv('cleaned_data.csv', index=False) # 不需要保存索引

<class 'pandas.core.frame.DataFrame'>
Int64Index: 530691 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    530691 non-null  object        
 1   StockCode    530691 non-null  object        
 2   Description  530691 non-null  object        
 3   Quantity     530691 non-null  int64         
 4   InvoiceDate  530691 non-null  datetime64[ns]
 5   UnitPrice    530691 non-null  float64       
 6   CustomerID   530691 non-null  object        
 7   Country      530691 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 36.4+ MB
